In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport bitnotes
%aimport watch_point
%aimport cluster
%aimport constants
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from bitnotes import BitNotes
from pprint import pprint
import math
import constants as const

### Load phrase base

In [2]:
import pickle

def load_phrase_base(file_name: str) -> (dict, list):
    with open(file_name, 'rb') as f:
        data = pickle.load(f)
        phrase_base = data.get('phrase_base', {})
        marks = data.get('marks', {})
        return phrase_base, marks
    
# phrase_base, marks = load_phrase_base('./data/texts/prob_text_base.pickle')
phrase_base, marks = load_phrase_base('./data/texts/lang_base_onebit.pickle')
# phrase_base, marks = load_phrase_base('./data/texts/monotone_base.pickle')
# phrase_base, marks = load_phrase_base('./data/texts/dna_base.pickle')

In [3]:
marks

{(0,): 'bel',
 (1,): 'blg',
 (2,): 'eng',
 (3,): 'epo',
 (4,): 'jbo',
 (5,): 'pol',
 (6,): 'rus',
 (7,): 'ukr'}

### Draw notations

In [ ]:
from ipythonblocks import BlockGrid

def draw_notation(notation: np.array):
    bit_grid = BlockGrid(len(notation), 1, fill=(17, 41, 129))
    for block in range(bit_grid.width):
        color = bit_grid[0, block]
        if notation[block]:
            bit_grid[0, block] = (244, 195, 173)
    bit_grid.lines_on = False
    bit_grid.show()

def draw_note_notations(notes, note_idx: int):
    for i in range(len(notes[note_idx])):
        draw_notation(notes.note_notation_as_bits(note_idx, i))

In [ ]:
abc_notes = BitNotes(note_count=26, 
                     notation_count=5, 
                     active_bits=8, 
                     bit_count=255)

key = list(phrase_base.keys())[0]
phrase = phrase_base[key][0]
bit_chord = abc_notes.phrase_chord(phrase)
draw_notation(abc_notes.notation_as_bits(bit_chord))
print(phrase)
for note_idx, notation_idx in phrase:
    notation = abc_notes.note_notation_as_bits(note_idx, notation_idx)
    draw_notation(notation)

In [ ]:
feeder = PhraseFeeder(phrase_base, marks)    

In [ ]:
phrases, output_bits = feeder.take_phrase_bunch()
print(output_bits, marks[output_bits])
print(phrases)

### Learn model

In [7]:
def feed_phrase_batch(cxnn: ContextNN, feeder: PhraseFeeder, bit_notes: BitNotes, count=200):
    phrases, output_bits = feeder.take_phrase_batch(count=200, random_bit_key=True)
    mark = feeder.marks[output_bits]    
    output_bits = set(output_bits)
    for phrase in phrases:
        bit_chord = bit_notes.phrase_chord_as_bits(phrase)
        cxnn.receive_bits(input_bits=bit_chord, output_bits=output_bits)
    return mark    

def feed_n_batches(cxnn: ContextNN, feeder: PhraseFeeder, bit_notes: BitNotes, n=10, batch_size=200):
    for i in range(n):
        mark = feed_phrase_batch(cxnn, feeder, bit_notes, count=batch_size)
        print(f'batch {i+1}/{n} {mark}')
    print(f'clusters: {cxnn.cluster_count()}')
        
def learn_cycle(cxnn: ContextNN, 
                feeder: PhraseFeeder, 
                bit_notes: BitNotes,
                learn_batch_size=200,
                consolidate=True,
                consolidate_batch_size=10,
                reduce=True,
                n_accumulate_batches=30,
                n_consolidate_batches=300,
                reduce_min_component=0.2,
                reduce_min_activations=100):
    print('\naccumulating clusters...')
    cxnn.state = const.STATE_ACCUMULATE
    feed_n_batches(cxnn, feeder, bit_notes, n=n_accumulate_batches, batch_size=learn_batch_size)
    
    if consolidate:
        print('\nconsolidating clusters...')
        cxnn.state = const.STATE_CONSOLIDATE
        feed_n_batches(cxnn, feeder, bit_notes, n=n_consolidate_batches, batch_size=consolidate_batch_size)
    
    if reduce:
        print('\nreducing clusters...')
        cxnn.reduce_clusters(min_component=reduce_min_component, 
                             min_activations=reduce_min_activations,
                             trim=True,
                             remain_parts=3,
                             clear_stats=True,
                             consolidate=True,
                             amnesty=True)
        print(f'clusters: {cxnn.cluster_count()}')
    

In [14]:
notes = bitnotes.BitNotes(note_count=26, 
                          notation_count=5, 
                          active_bits=8, 
                          bit_count=256)

feeder = phrase_feeder.PhraseFeeder(phrase_base, marks)  

cxnn = context_nn.ContextNN(input_bit_count=256,
                            output_bit_count=len(marks.keys()),
                            watch_point_count=2000,
                            watch_bit_count=32,
                            cluster_make_threshold=6,
                            cluster_activate_threshold=4,
                            bit_notes=notes,
                            data_marks=marks)

In [17]:
%%time
for n in range(4):
    print(f'\nCycle {n+1}')
    learn_cycle(cxnn, 
                feeder, 
                notes,
                learn_batch_size=200,
                consolidate=True,
                consolidate_batch_size=10,
                reduce=True,
                n_accumulate_batches=32,
                n_consolidate_batches=80,
                reduce_min_component=0.10,
                reduce_min_activations=50)


Cycle 1

accumulating clusters...
batch 1/32 bel
batch 2/32 epo
batch 3/32 jbo
batch 4/32 blg
batch 5/32 ukr
batch 6/32 rus
batch 7/32 pol
batch 8/32 eng
batch 9/32 epo
batch 10/32 rus
batch 11/32 bel
batch 12/32 pol
batch 13/32 blg
batch 14/32 jbo
batch 15/32 eng
batch 16/32 ukr
batch 17/32 pol
batch 18/32 eng
batch 19/32 rus
batch 20/32 ukr
batch 21/32 blg
batch 22/32 epo
batch 23/32 bel
batch 24/32 jbo
batch 25/32 ukr
batch 26/32 jbo
batch 27/32 bel
batch 28/32 eng
batch 29/32 epo
batch 30/32 blg
batch 31/32 pol
batch 32/32 rus
clusters: 361024

consolidating clusters...
batch 1/80 epo
batch 2/80 eng
batch 3/80 rus
batch 4/80 pol
batch 5/80 bel
batch 6/80 jbo
batch 7/80 blg
batch 8/80 ukr
batch 9/80 epo
batch 10/80 eng
batch 11/80 blg
batch 12/80 jbo
batch 13/80 bel
batch 14/80 rus
batch 15/80 ukr
batch 16/80 pol
batch 17/80 bel
batch 18/80 jbo
batch 19/80 rus
batch 20/80 ukr
batch 21/80 blg
batch 22/80 eng
batch 23/80 epo
batch 24/80 pol
batch 25/80 rus
batch 26/80 eng
batch 27/80

### Model stats

In [18]:
print(cxnn.cluster_count())
print(sorted(cxnn.point_stats(), key=lambda x: x[0]))
# print(cxnn.point_stats())
marks

57766
[(0, 1), (0, 89), (0, 19), (0, 4), (0, 43), (0, 1), (0, 0), (0, 17), (0, 12), (0, 34), (0, 5), (0, 7), (0, 0), (0, 42), (0, 6), (0, 45), (0, 22), (0, 27), (0, 34), (0, 4), (0, 40), (0, 13), (0, 7), (0, 13), (0, 3), (0, 54), (0, 37), (0, 10), (0, 0), (0, 17), (0, 7), (0, 29), (0, 16), (0, 14), (0, 2), (0, 16), (0, 24), (0, 6), (0, 10), (0, 6), (0, 17), (0, 44), (0, 32), (0, 2), (0, 16), (0, 26), (0, 0), (0, 8), (0, 6), (0, 11), (0, 4), (0, 11), (0, 52), (0, 15), (0, 58), (0, 23), (0, 1), (0, 37), (0, 8), (0, 2), (0, 13), (0, 0), (0, 10), (0, 24), (0, 14), (0, 4), (0, 15), (0, 9), (0, 0), (0, 1), (0, 12), (0, 5), (0, 46), (0, 0), (0, 112), (0, 35), (0, 6), (0, 6), (0, 0), (0, 30), (0, 17), (0, 53), (0, 2), (0, 8), (0, 4), (0, 18), (0, 0), (0, 0), (0, 3), (0, 25), (0, 6), (0, 16), (0, 24), (0, 19), (0, 17), (0, 2), (0, 11), (0, 0), (0, 13), (0, 21), (0, 8), (0, 12), (0, 41), (0, 11), (0, 28), (0, 11), (0, 6), (0, 0), (0, 26), (0, 5), (0, 35), (0, 7), (0, 2), (0, 11), (0, 38), (0, 9)

{(0,): 'bel',
 (1,): 'blg',
 (2,): 'eng',
 (3,): 'epo',
 (4,): 'jbo',
 (5,): 'pol',
 (6,): 'rus',
 (7,): 'ukr'}

In [14]:
print(cxnn.cluster_count())
pprint(cxnn.cluster_len_stats())
pprint(cxnn.cluster_activity_stats())

78271
{6: 215, 7: 12951, 8: 62434, 9: 2586, 10: 77, 11: 8}
{0: 78271}


In [15]:
cxnn.cluster_consolidated_stats()

{0: 5231, 1: 20277, 2: 15368, 3: 14145, 4: 11641, 5: 11609}

In [ ]:
def dict_value(d: dict, n=0) -> object:
    i = 0
    diter = iter(d)
    while i <= n:
        key = next(diter)
        i += 1
    return d[key]


point = cxnn.point_objects[8]
cluster_idx = 0
cluster = point.cluster_objects[cluster_idx]
pprint(cluster.stats)
print(sum(cluster.stats.values()))
# pprint(cluster.component_stats())
# pprint(cluster.bit_rate())

print(sorted(cluster.component_stats().items(), key=lambda x: x[1], reverse=True))
print(point.cluster_masks)
print(np.where(cluster.bit_mask > 0)[0])
print(point.watch_bits)
print(point.cluster_masks[cluster_idx])
print(len(point.cluster_masks), len(point.cluster_objects))

# parts = cluster.component_s
# enthropy = 0.0
# for part in parts.values():
#     enthropy += -part * math.log2(part)
# print(enthropy)

In [ ]:
cluster_with_stats = None
for wp in cxnn.point_objects:
    for cluster in wp.cluster_objects:    
        if cluster.stats:
            cluster_with_stats = cluster
            break
    else: 
        continue
    break    

if cluster_with_stats:
    print(cluster_with_stats.stats, cluster_with_stats.consolidated)
else:
    print('nothing to print')
    
    

### Save-load model

In [20]:
import pickle

def save_cxnn(cxnn: ContextNN, file_name: str):
    with open(file_name, 'wb') as f:
        pickle.dump(cxnn, f)

def load_cxnn(file_name: str) -> ContextNN:
    with open(file_name, 'rb') as f:
        cxnn = pickle.load(f)
        return cxnn
    
def save_notes(notes: BitNotes, file_name: str):
    with open(file_name, 'wb') as f:
        pickle.dump(notes, f)

def load_notes(file_name: str) -> BitNotes:
    with open(file_name, 'rb') as f:
        notes = pickle.load(f)
        return notes



In [21]:
save_cxnn(cxnn, './data/cxnn_langs.pickle')

In [ ]:
# cxnn = load_cxnn('./data/cxnn_langs.pickle')

In [ ]:
save_cxnn(cxnn, './data/cxnn_monotone.pickle')

In [9]:
save_cxnn(cxnn, './data/cxnn_probs.pickle')

In [18]:
save_cxnn(cxnn, './data/cxnn_dna.pickle')